### 1-Strength Build Optimization: Class Selection, Stat Allocation, and Weapon Recommendation


Let's solve the Strength build optimization with data!

Given a target level of 150, we want to determine:

- Which starting class a hero should pick.
- How their stats should be allocated to maximize Strength efficiency while respecting key constraints.
- Which weapons will make the build truly devastating, considering only the **highest total attack version** per weapon that the hero can actually equip.

Constraints / Considerations:

1- **Strength**: primary stat, aim up to **80 points**.  
2- **Vigor**: secondary stat for survivability, allocate up to **60 points**.  
3- **Endurance**: secondary stat affecting stamina and equip load, allocate up to **50 points**.  
4- **Other stats** (Mind, Dexterity, Intelligence, Faith, Arcane): generally not increased for pure Strength builds; points here are wasted.  
5- **Minimize wasted points**: allocate as much as possible to Strength, Vigor, and Endurance only.  
6- **Weapons**: choose only those with **S or A Strength scaling**, **within the hero’s Strength, Dexterity, Intelligence, Faith, and Arcane limits**, and only the **highest upgrade per weapon**.

> Note: With only 10 starting classes we could eyeball the hero class, but using this optimizer scales effortlessly if the number of classes is very large.


In [0]:
%sql
SELECT * FROM eldenringcatalog.gold.startingclasses

Class_Name,Level,Vigor,Mind,Endurance,Strength,Dexterity,Intelligence,Faith,Arcane
Vagabond,9,15,10,11,14,13,9,9,7
Warrior,8,11,12,11,10,16,10,8,9
Hero,7,14,9,12,16,9,7,8,11
Bandit,5,10,11,10,9,13,9,8,14
Astrologer,6,9,15,9,8,12,16,7,9
Prophet,7,10,14,8,11,10,7,16,10
Samurai,9,12,11,13,12,15,9,8,8
Prisoner,9,11,12,11,11,14,14,6,9
Confessor,10,10,13,10,12,12,9,14,9
Wretch,1,10,10,10,10,10,10,10,10



Elden Ring offers 10 starting classes. Each class begins at a specific level and comes with its own distribution of attributes such as Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, and Arcane.


Let’s build the StrengthBuildOptimizer class to tackle our problem: given a target level of 150, it will determine which starting class our hero should pick, allocate stats to maximize Strength efficiency while respecting key constraints, minimize wasted points in irrelevant attributes, and recommend the highest upgrade version of weapons with S or A Strength scaling that the hero can equip based on Strength, Dexterity, Intelligence, Faith, and Arcane stats

#### **StrengthBuildOptimizer class**

In [0]:
import logging
logger = logging.getLogger(__name__)

class StrengthBuildOptimizer:

    def __init__(self, spark):
        self.spark = spark

    # Load starting classes
    def _load_classes(self):
        df = self.spark.sql("SELECT * FROM eldenringcatalog.gold.startingclasses")
        # Normalize columns to lowercase
        for col in df.columns:
            df = df.withColumnRenamed(col, col.lower())
        return df.collect()

    # Choose best starting class for PURE STR build
    def get_best_str_class(self):
        classes = self._load_classes()

        useful = ["strength", "vigor", "endurance"]
        ranked = []

        for c in classes:
            base_stats = {
                "vigor": c.vigor,
                "mind": c.mind,
                "endurance": c.endurance,
                "strength": c.strength,
                "dexterity": c.dexterity,
                "intelligence": c.intelligence,
                "faith": c.faith,
                "arcane": c.arcane
            }

            useful_sum = sum(base_stats[s] for s in useful)
            wasted_sum = sum(v for k, v in base_stats.items() if k not in useful)
            efficiency = useful_sum - wasted_sum

            ranked.append({
                "class_name": c.class_name,
                "starting_level": c.level,
                "base_stats": base_stats,
                "efficiency_score": efficiency
            })

        best = sorted(ranked, key=lambda r: r["efficiency_score"], reverse=True)[0]
        logger.info(f"🏆 Best class for PURE STR = {best['class_name']}")
        return best

    # Stat allocation for PURE STR ONLY
    def allocate_str_stats(self, base, available_points):

        soft_caps = {
            "vigor": 60,
            "endurance": 50,
            "strength": 80
        }

        ratios = {
            "strength": 0.50,
            "vigor": 0.30,
            "endurance": 0.20
        }

        stats = base.copy()

        for stat, pct in ratios.items():
            stats[stat] = int(min(soft_caps[stat], stats[stat] + available_points * pct))

        return stats


    # FINAL PURE STR BUILD OPTIMIZATION
    def optimize_pure_str_build(self, target_level: int = 150):

        # 1) Best STR class
        best_class = self.get_best_str_class()
        base_stats = best_class["base_stats"]
        starting_level = best_class["starting_level"]
        available_points = target_level - starting_level

        # 2) Allocate stats
        final_stats = self.allocate_str_stats(base_stats, available_points)

        # 3) Recommended STR weapons
        df_items = self.spark.sql(f"""
                        SELECT weapon_name, category, upgrade_level, total_attack_power,
                            attack_physical, attack_magic, attack_fire, attack_lightning, attack_holy,
                            scaling_strength, scaling_dexterity
                        FROM (
                            SELECT w.weapon_name, w.category, ws.upgrade_level, ws.total_attack_power,
                                ws.attack_physical, ws.attack_magic, ws.attack_fire, ws.attack_lightning, ws.attack_holy,
                                ws.scaling_strength, ws.scaling_dexterity,
                                ROW_NUMBER() OVER (PARTITION BY w.weapon_name ORDER BY ws.total_attack_power DESC) as rn
                            FROM eldenringcatalog.gold.dim_weapons w
                            JOIN eldenringcatalog.gold.fact_weapon_stats ws
                                ON w.weapon_name = ws.weapon_name
                            WHERE ws.scaling_strength IN ('S', 'A')
                            AND w.required_strength <= {final_stats['strength']}
                            AND w.required_intelligence <= {final_stats['intelligence']}
                            AND w.required_faith <= {final_stats['faith']}
                            AND w.required_arcane <= {final_stats['dexterity']}
                            AND w.required_dexterity <= {final_stats['dexterity']}
                        ) t
                        WHERE rn = 1
                        ORDER BY total_attack_power DESC
                        LIMIT 10
                    """)

        return {
            "target_level": target_level,
            "starting_class": best_class["class_name"],
            "starting_level": starting_level,
            "base_stats": base_stats,
            "optimized_stats": final_stats,
            "recommended_items": df_items
        }


In [0]:
optimizer = StrengthBuildOptimizer(spark)
result = optimizer.optimize_pure_str_build(target_level=150)

# Display summary
print(f"\n🏹 Strength Build Summary (Target Level: {result['target_level']})")
print(f"Starting Class: {result['starting_class']} (Level {result['starting_level']})\n")

# Base stats
print("Base Stats:")
for stat, value in result['base_stats'].items():
    print(f"  {stat.capitalize():<12}: {value}")

# Optimized stats
print("\nOptimized Stats:")
for stat, value in result['optimized_stats'].items():
    print(f"  {stat.capitalize():<12}: {value}")

# Recommended weapons
print("\nTop Recommended STR Weapons:")
display(result['recommended_items'])


🏹 Strength Build Summary (Target Level: 150)
Starting Class: Hero (Level 7)

Base Stats:
  Vigor       : 14
  Mind        : 9
  Endurance   : 12
  Strength    : 16
  Dexterity   : 9
  Intelligence: 7
  Faith       : 8
  Arcane      : 11

Optimized Stats:
  Vigor       : 56
  Mind        : 9
  Endurance   : 40
  Strength    : 80
  Dexterity   : 9
  Intelligence: 7
  Faith       : 8
  Arcane      : 11

Top Recommended STR Weapons:


weapon_name,category,upgrade_level,total_attack_power,attack_physical,attack_magic,attack_fire,attack_lightning,attack_holy,scaling_strength,scaling_dexterity
Giant-Crusher,Colossal Weapons,Cold +25,473.0,270.0,203.0,0.0,0.0,0.0,A,-
Warped Axe,Axes,Cold +25,378.0,216.0,162.0,0.0,0.0,0.0,A,-
Gargoyle's Great Axe,Greataxes,Cold +25,374.0,214.0,160.0,0.0,0.0,0.0,A,E
Brick Hammer,Great Hammers,Cold +25,372.0,212.0,160.0,0.0,0.0,0.0,A,-
Greataxe,Greataxes,Heavy +25,343.0,343.0,0.0,0.0,0.0,0.0,A,-
Prelate's Inferno Crozier,Colossal Weapons,Heavy +25,332.0,332.0,0.0,0.0,0.0,0.0,A,-
Longhaft Axe,Greataxes,Heavy +25,331.0,331.0,0.0,0.0,0.0,0.0,A,-
Executioner's Greataxe,Greataxes,Heavy +25,319.0,319.0,0.0,0.0,0.0,0.0,A,-
Rusted Anchor,Greataxes,Heavy +25,312.0,312.0,0.0,0.0,0.0,0.0,A,-
Battle Axe,Axes,Heavy +25,286.0,286.0,0.0,0.0,0.0,0.0,A,-


# 2-What weapon should I choose at level 1 that will still be viable at level 150 (Strenght Build) ?

From our analysis, the **Hero** class emerges as the best starting choice for a Strength build, as it minimizes wasted stat points.  

**Base stats for the Hero class:**

- Vigor       : 14  
- Mind        : 9  
- Endurance   : 12  
- Strength    : 16  
- Dexterity   : 9  
- Intelligence: 7  
- Faith       : 8  
- Arcane      : 11  

**Optimized stats at target level 150:**

- Vigor       : 56  
- Mind        : 9  
- Endurance   : 40  
- Strength    : 80  
- Dexterity   : 9  
- Intelligence: 7  
- Faith       : 8  
- Arcane      : 11


so level-1 weapon constraints become 
required_strength  <= 16 &
required_dexterity <= 9

at level 150 with strength build :
Strength : 80 and Dexterity : 9

To remain strong late game, the weapon must have: scaling_strength IN ('A', 'S')


In [0]:
%sql
-- Step 1: Start from hero class stats
WITH hero_stats AS (
    SELECT 16 AS hero_strength, 9 AS hero_dexterity
)

SELECT 
    w.weapon_id,
    w.weapon_name,
    w.required_strength,
    w.required_dexterity,
    fs.upgrade_level,
    fs.scaling_strength,
    fs.total_attack_power,
    fs.attack_physical,
    fs.attack_magic,
    fs.attack_fire,
    fs.attack_lightning
FROM eldenringcatalog.gold.dim_weapons w
CROSS JOIN hero_stats h
JOIN eldenringcatalog.gold.fact_weapon_stats fs 
    ON w.weapon_name = fs.weapon_name
WHERE 
    -- Hero can equip at level 1
    w.required_strength <= h.hero_strength
    AND w.required_dexterity <= h.hero_dexterity

    -- Good endgame scaling
    AND fs.scaling_strength IN ('A', 'S')

    -- Ensure we select the MAX upgrade level version
    AND fs.total_attack_power = (
    SELECT MAX(fs2.total_attack_power)
    FROM eldenringcatalog.gold.fact_weapon_stats fs2
    WHERE fs2.weapon_name = w.weapon_name
)

ORDER BY 
    fs.scaling_strength DESC,   -- S first
    w.required_strength ASC;    -- lower requirements preferred


weapon_id,weapon_name,required_strength,required_dexterity,upgrade_level,scaling_strength,total_attack_power,attack_physical,attack_magic,attack_fire,attack_lightning
278,Serpent-Hunter,0,0,Standard +10,A,271.0,271.0,0.0,0.0,0.0
405,Lamenting Visage,9,0,Standard +10,A,173.0,173.0,0.0,0.0,0.0


Serpent-Hunter cannot be obtained at the beginning of the game. It is only found on a corpse to the left as you enter Rykard, Lord of Blasphemy’s boss arena. As a result, its required Strength and Dexterity values should not be considered for early-game analysis, and the data currently associated with Serpent-Hunter appears to be incorrect and needs to be corrected.
Given these constraints, **Lamenting Visage** remains the only valid candidate that satisfies our criteria.

![Lamenting Visage](https://static0.gamerantimages.com/wordpress/wp-content/uploads/2024/07/elden-ring-lamenting-visage-stats.jpg?q=49&fit=contain&w=750&h=422&dpr=2)